# Fire detection

## Imports

### Libraries

In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision import models

import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

### Local

In [2]:
from src.dataset import FireImageDataset

## Data

### Loading

In [19]:
# Define the data directories
DATA_DIR = os.path.join(os.getcwd(), 'data')
IMAGE_DIR = os.path.join(DATA_DIR, 'images')
LABEL_DIR = os.path.join(DATA_DIR, 'labels')

TRAIN_IMAGE_DIR = os.path.join(IMAGE_DIR, 'train')
TRAIN_LABEL_DIR = os.path.join(LABEL_DIR, 'train')
VAL_IMAGE_DIR = os.path.join(IMAGE_DIR, 'val')
VAL_LABEL_DIR = os.path.join(LABEL_DIR, 'val')

# Initialize the datasets
train_set = FireImageDataset(TRAIN_IMAGE_DIR, TRAIN_LABEL_DIR)
validation_set = FireImageDataset(VAL_IMAGE_DIR, VAL_LABEL_DIR)

# Verify that the image and label files are correctly paired
train_set.verify()
validation_set.verify()

### Exploration

In [8]:
# FIXME

### Preprocessing

In [9]:
# FIXME

### Augmentation

In [10]:
# FIXME

## Model(s)

### Architecture

In [11]:
# FIXME

### Loading

In [12]:
# FIXME

### Fine-tuning and training

In [13]:
# FIXME

### Evaluation

In [14]:
# FIXME

## Explanability

In [15]:
# FIXME